# Aerospike Spark Connector Tutorial for Scala

## Tested with Java 8, Spark 2.4.0, Python 3.7,  Scala 2.11.12, and  Spylon ( https://pypi.org/project/spylon-kernel/)


In [15]:
%%init_spark 
launcher.jars = ["aerospike-spark-assembly-2.5.0.jar"] 
launcher.master = "local[*]"

In [16]:
//Specify the Seed Host of the Aerospike Server
val AS_HOST =" 172.16.39.143:3000"

AS_HOST: String = " 172.16.39.143:3000"


In [17]:
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SaveMode

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SaveMode


## Schema in the Spark Connector

-  Aerospike is schemaless, however spark adher to schema. After the schema is decided upon (either through inference or given), data within the bins must honor the types. 

- To infer the schema, the connector samples a set of records (configurable through `aerospike.schema.scan`) to decide the name of bins/columns and their types. This implies that the derived schema depends entirely upon sampled records.  

- **Note that `__key` was not part of provided schema. So how can one query using `__key`? We can just add `__key` in provided schema with appropriate type. Similarly we can add `__gen` or `__ttl` etc.**  
         
      val schemaWithPK: StructType = new StructType(Array(
                StructField("__key",IntegerType, nullable = false),    
                StructField("id", IntegerType, nullable = false),
                StructField("name", StringType, nullable = false),
                StructField("age", IntegerType, nullable = false),
                StructField("salary",IntegerType, nullable = false)))
                
- **We recommend that you provide schema for queries that involve complex data types such as lists, maps, and mixed types. Using schema inference for CDT may cause unexpected issues.** 
          

### Flexible schema inference 

Spark assumes that the underlying data store (Aerospike in this case) follows a strict schema for all the records within a table. However, Aerospike is a No-SQL DB and is schemaless. Hence a single bin (mapped to a column ) within a set ( mapped to a table ) could technically hold values of multiple Aerospike supported types. The Spark connector reconciles this incompatibility with help of certain rules. Please choose the configuration that suits your use case. The strict configuration (aerospike.schema.flexible = false ) could be used when you have modeled your data in Aerospike to adhere to a strict schema i.e. each record within the set has the same schema.


#### aerospike.schema.flexible = true (default) 
   
  If none of the column types in the user-specified schema match the bin types of a record in Aerospike, a record with NULLs is returned in the result set. 

Please use the filter() in Spark to filter out NULL records. For e.g. df.filter("gender == NULL").show(false), where df is a dataframe and gender is a field that was not specified in the user-specified schema. 

If the above mismatch is limited to fewer columns in the user-specified schema then NULL would be returned for those columns in the result set. **Note: there is no way to tell apart a NULL due to missing value in the original data set and the NULL due to mismatch, at this point. Hence, the user would have to treat all NULLs as missing values.** The columns that are not a part of the schema will be automatically filtered out in the result set by the connector.

Please note that if any field is set to NOT nullable i.e. nullable = false, your query will error out if there’s a type mismatch between an Aerospike bin and the column type specified in the user-specified schema.
  

### Create sample data to demonstrate flexible schema inference

In [18]:
import com.aerospike.client.policy.WritePolicy
import com.aerospike.spark.sql.AerospikeConnection
import org.apache.spark.sql.SparkSession
import com.aerospike.client.{AerospikeClient, AerospikeException, Bin, Key}

val conf = sc.getConf.clone();
conf.set("aerospike.seedhost" , AS_HOST)
conf.set("aerospike.schema.flexible" , "true") //by default it is always true
val spark1= SparkSession.builder().config(conf).master("local[2]").getOrCreate()
val client = AerospikeConnection.getClient(conf)
val flexsetname = "flexschema"
val wp = new WritePolicy()
    wp.expiration = 600 // expire data in 10 minutes
    for (i <- 1 to 100) {
      val key = new Key("test", flexsetname, i)
      client.delete(null, key )
      if( i %2 ==0){
        client.put(wp, key, new Bin("one", i.toInt), new Bin("two", i.toInt))
      }else{
        client.put(wp, key, new Bin("one", i.toInt), new Bin("two", i.toString))
      }
    }


val flexibleSchema= StructType (
      Seq(
        StructField("one", IntegerType, true ),
        StructField("two", IntegerType, true )
      )
    )
spark1.sqlContext.read.format("com.aerospike.spark.sql")
.option("aerospike.seedhost", AS_HOST)
.option("aerospike.keyPath", "/etc/aerospike/features.conf") //Path to feature file, while running in cluster this file needs to be on all drivers. Consult documentation on how to read from HDFS or as string. 
.option("aerospike.set", flexsetname)
.option("aerospike.namespace", "test")
.schema(flexibleSchema).load().show()
spark1.close()
//Please note that, in case of type mismatch all columns with odd value of `one`(which had string type) is set to null

+---+----+
|one| two|
+---+----+
| 82|  82|
| 67|null|
| 29|null|
| 39|null|
| 16|  16|
| 34|  34|
|  1|null|
| 77|null|
| 52|  52|
| 27|null|
| 25|null|
| 11|null|
| 15|null|
| 96|  96|
| 97|null|
|  4|   4|
| 89|null|
| 14|  14|
| 79|null|
| 71|null|
+---+----+
only showing top 20 rows



import com.aerospike.client.policy.WritePolicy
import com.aerospike.spark.sql.AerospikeConnection
import org.apache.spark.sql.SparkSession
import com.aerospike.client.{AerospikeClient, AerospikeException, Bin, Key}
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@75dda421
spark1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5bc82eb8
client: com.aerospike.client.AerospikeClient = com.aerospike.client.AerospikeClient@88e1ccf
flexsetname: String = flexschema
wp: com.aerospike.client.policy.WritePolicy = com.aerospike.client.policy.WritePolicy@67b51744
wp.expiration: Int = 600
flexibleSchema: org.apache.spark.sql.types.StructType = StructType(StructField(one,IntegerType,true), StructField(two,IntegerType,true))


#### aerospike.schema.flexible = false 

If a mismatch between the user-specified schema and the schema of a record in Aerospike is detected at the bin/column level, your query will error out.


In [19]:
//When strict matching is set, we will get an exception due to type mismatch with schema provided.
import scala.util.Try

val strictConf = conf.clone();

val df = Try{

    strictConf.set("aerospike.schema.flexible" , "false")
    val spark2 = SparkSession.builder().config(strictConf).master("local[2]").getOrCreate()
    spark2.sqlContext.read.
    format("com.aerospike.spark.sql").
    option("aerospike.seedhost", AS_HOST).
    option("aerospike.keyPath", "/etc/aerospike/features.conf").
    option("aerospike.set", flexsetname).
    option("aerospike.namespace", "test").
    schema(flexibleSchema).load().show()
    spark2.close()
    }



             

    

import scala.util.Try
strictConf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@42caf17c
df: scala.util.Try[Unit] =
Failure(org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 2.0 failed 1 times, most recent failure: Lost task 6.0 in stage 2.0 (TID 11, localhost, executor driver): com.aerospike.spark.sql.TypeConverter$TypeMismatchException
	at com.aerospike.spark.sql.TypeConverter$.matchesSchemaType$1(TypeConverter.scala:158)
	at com.aerospike.spark.sql.TypeConverter$.binToValue(TypeConverter.scala:190)
	at com.aerospike.spark.sql.RowIterator$$anonfun$29.apply(KeyRecordRDD.scala:701)
	at com.aerospike.spark.sql.RowIterator$$anonfun$29.apply(KeyRecordRDD.scala:694)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234...

## Create sample data and write it into Aerospike Database

In [20]:
//Create test data

val num_records=1000
val rand = scala.util.Random

//schema of input data
val spark = SparkSession.builder().config(strictConf).master("local[*]").getOrCreate()
val schema: StructType = new StructType(
    Array(
    StructField("id", IntegerType, nullable = false),
    StructField("name", StringType, nullable = false),
    StructField("age", IntegerType, nullable = false),
    StructField("salary",IntegerType, nullable = false)
  ))

val inputDF = {
    val inputBuf=  new ArrayBuffer[Row]()
    for ( i <- 1 to num_records){
        val name = "name"  + i
        val age = i%100
        val salary = 50000 + rand.nextInt(50000)
        val id = i 
        val r = Row(id, name, age,salary)
        inputBuf.append(r)
    }
    val inputRDD = spark.sparkContext.parallelize(inputBuf.toSeq)
    spark.createDataFrame(inputRDD,schema)
}

inputDF.show(10)

//Write the Sample Data to Aerospike
inputDF.write.mode(SaveMode.Overwrite) 
.format("com.aerospike.spark.sql") //aerospike specific format
.option("aerospike.seedhost", AS_HOST) //db hostname, can be added multiple hosts, delimited with ":"
.option("aerospike.namespace", "test") //use this namespace 
.option("aerospike.writeset", "input_data") //write to this set
.option("aerospike.updateByKey", "id") //indicates which columns should be used for construction of primary key
.save()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1| name1|  1| 86233|
|  2| name2|  2| 93303|
|  3| name3|  3| 65431|
|  4| name4|  4| 55790|
|  5| name5|  5| 61242|
|  6| name6|  6| 72064|
|  7| name7|  7| 55726|
|  8| name8|  8| 96412|
|  9| name9|  9| 78500|
| 10|name10| 10| 55249|
+---+------+---+------+
only showing top 10 rows



num_records: Int = 1000
rand: util.Random.type = scala.util.Random$@7cd61e0
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4b89385f
schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,false), StructField(name,StringType,false), StructField(age,IntegerType,false), StructField(salary,IntegerType,false))
inputDF: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


## Load data into a DataFrame without specifying any schema i.e. using connector schema inference

In [21]:
// Create a Spark DataFrame by using the Connector Schema inference mechanism

val loadedDFWithoutSchema=spark
.sqlContext
.read
.format("com.aerospike.spark.sql")
.option("aerospike.seedhost", AS_HOST)
.option("aerospike.keyPath", "/etc/aerospike/features.conf") //Path to feature file, while running in cluster this file needs to be on all drivers. Consult documentation on how to read from HDFS or as string. 
.option ("aerospike.namespace", "test")
.option("aerospike.set", "input_data") //read the data from this set
.load
loadedDFWithoutSchema.printSchema()
//Notice that schema of loaded data has some additional fields. 
// When connector infers schema, it also adds internal metadata.

root
 |-- __key: string (nullable = true)
 |-- __digest: binary (nullable = false)
 |-- __expiry: integer (nullable = false)
 |-- __generation: integer (nullable = false)
 |-- __ttl: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: long (nullable = true)
 |-- id: long (nullable = true)



loadedDFWithoutSchema: org.apache.spark.sql.DataFrame = [__key: string, __digest: binary ... 7 more fields]


## Load data into a DataFrame with user specified schema 

In [22]:
//Data can be loaded with known schema as well.
val loadedDFWithSchema=spark
.sqlContext
.read
.format("com.aerospike.spark.sql")
.schema(schema)
.option("aerospike.seedhost",AS_HOST)
.option("aerospike.featurekey", "/etc/aerospike/features.conf") 
.option ("aerospike.namespace", "test")
.option("aerospike.set", "input_data").load
loadedDFWithSchema.show(5)

+---+-------+---+------+
| id|   name|age|salary|
+---+-------+---+------+
|829|name829| 29| 65980|
|486|name486| 86| 74301|
|759|name759| 59| 60474|
|524|name524| 24| 99707|
|215|name215| 15| 94793|
+---+-------+---+------+
only showing top 5 rows



loadedDFWithSchema: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


## Writing Sample Complex Data Types (CDT) data into Aerospike

In [23]:
val complex_data_json="resources/nested_data.json"
val alias=  StructType(List(
    StructField("first_name",StringType, false),
    StructField("last_name",StringType, false)))

  val name= StructType(List(
    StructField("first_name",StringType, false),
    StructField("aliases",ArrayType(alias), false )
  ))

  val street_adress= StructType(List(
    StructField("street_name", StringType, false),
    StructField("apt_number" , IntegerType, false)))

  val address = StructType( List(
    StructField ("zip" , LongType, false),
    StructField("street", street_adress, false),
    StructField("city", StringType, false)))

  val workHistory = StructType(List(
    StructField ("company_name" , StringType, false),
    StructField( "company_address" , address, false),
    StructField("worked_from", StringType, false)))

  val person=  StructType ( List(
    StructField("name" , name, false, Metadata.empty),
    StructField("SSN", StringType, false,Metadata.empty),
    StructField("home_address", ArrayType(address), false),
    StructField("work_history", ArrayType(workHistory), false)))

val cmplx_data_with_schema=spark.read.schema(person).json(complex_data_json)

cmplx_data_with_schema.printSchema()
cmplx_data_with_schema.write.mode(SaveMode.Overwrite) 
.format("com.aerospike.spark.sql") //aerospike specific format
.option("aerospike.seedhost", AS_HOST) //db hostname, can be added multiple hosts, delimited with ":"
.option("aerospike.namespace", "test") //use this namespace 
.option("aerospike.writeset", "scala_complex_input_data") //write to this set
.option("aerospike.updateByKey", "name.first_name") //indicates which columns should be used for construction of primary key
.save()

root
 |-- name: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- aliases: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- first_name: string (nullable = true)
 |    |    |    |-- last_name: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- home_address: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- zip: long (nullable = true)
 |    |    |-- street: struct (nullable = true)
 |    |    |    |-- street_name: string (nullable = true)
 |    |    |    |-- apt_number: integer (nullable = true)
 |    |    |-- city: string (nullable = true)
 |-- work_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- company_name: string (nullable = true)
 |    |    |-- company_address: struct (nullable = true)
 |    |    |    |-- zip: long (nullable = true)
 |    |    |    |-- street: struct (nullable = true)
 |    |    |    |    

complex_data_json: String = resources/nested_data.json
alias: org.apache.spark.sql.types.StructType = StructType(StructField(first_name,StringType,false), StructField(last_name,StringType,false))
name: org.apache.spark.sql.types.StructType = StructType(StructField(first_name,StringType,false), StructField(aliases,ArrayType(StructType(StructField(first_name,StringType,false), StructField(last_name,StringType,false)),true),false))
street_adress: org.apache.spark.sql.types.StructType = StructType(StructField(street_name,StringType,false), StructField(apt_number,IntegerType,false))
address: org.apache.spark.sql.types.StructType = StructType(StructField(zip,LongType,false), StructField(street,StructType(StructField(street_name,StringType,false), StructField(apt_number,IntegerType,false)),fal...

## Load Complex Data Types (CDT) into a DataFrame with user specified schema

In [24]:
val loadedComplexDFWithSchema=spark
.sqlContext
.read
.format("com.aerospike.spark.sql")
.option("aerospike.seedhost", AS_HOST)
.option("aerospike.keyPath", "/etc/aerospike/features.conf") //Path to feature file, while running in cluster this file needs to be on all drivers. Consult documentation on how to read from HDFS or as string. 
.option ("aerospike.namespace", "test")
.option("aerospike.set", "scala_complex_input_data") //read the data from this set
.schema(person)
.load
loadedComplexDFWithSchema.printSchema()
//Please note the difference in types of loaded data in both cases. With schema, we extactly infer complex types.

root
 |-- name: struct (nullable = false)
 |    |-- first_name: string (nullable = false)
 |    |-- aliases: array (nullable = false)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- first_name: string (nullable = false)
 |    |    |    |-- last_name: string (nullable = false)
 |-- SSN: string (nullable = false)
 |-- home_address: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- zip: long (nullable = false)
 |    |    |-- street: struct (nullable = false)
 |    |    |    |-- street_name: string (nullable = false)
 |    |    |    |-- apt_number: integer (nullable = false)
 |    |    |-- city: string (nullable = false)
 |-- work_history: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- company_name: string (nullable = false)
 |    |    |-- company_address: struct (nullable = false)
 |    |    |    |-- zip: long (nullable = false)
 |    |    |    |-- street: struct (nullable = false)
 |  

loadedComplexDFWithSchema: org.apache.spark.sql.DataFrame = [name: struct<first_name: string, aliases: array<struct<first_name:string,last_name:string>>>, SSN: string ... 2 more fields]


# Quering Aerospike Data using SparkSQL

### Things to keep in mind
   1. Queries that involve Primary Key in the predicate trigger aerospike_batch_get()( https://www.aerospike.com/docs/client/c/usage/kvs/batch.html) and run extremely fast. For e.g. a query containing `__key` with, with no `OR` between two bins.
   2. All other queries may entail a full scan of the Aerospike DB if they can’t be converted to Aerospike batchget. 

## Queries that include Primary Key in the Predicate

In case of batchget queries we can also apply filters upon metadata columns like `__gen` or `__ttl` etc. To do so, these columns should be exposed through schema (if schema provided). 

In [25]:
val batchGet1= spark.sqlContext
.read
.format("com.aerospike.spark.sql")
.option("aerospike.seedhost", AS_HOST)
.option("aerospike.featurekey", "/etc/aerospike/features.conf") 
.option ("aerospike.namespace", "test")
.option("aerospike.set", "input_data")
.option("aerospike.keyType", "int") //used to hint primary key(PK) type when schema is not provided.
.load.where("__key = 829")
batchGet1.show()
//Please be aware Aerospike database supports only equality test with PKs in primary key query. 
//So, a where clause with "__key >10", would result in scan query!

+-----+--------------------+--------+------------+-----+-------+---+------+---+
|__key|            __digest|__expiry|__generation|__ttl|   name|age|salary| id|
+-----+--------------------+--------+------------+-----+-------+---+------+---+
|  829|[00 B0 3B 5E BD 9...|       0|           1|   -1|name829| 29| 65980|829|
+-----+--------------------+--------+------------+-----+-------+---+------+---+



batchGet1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [__key: int, __digest: binary ... 7 more fields]


In [26]:
//In this query we are doing *OR* between PK subqueries 

val somePrimaryKeys= 1.to(10).toSeq
val someMoreKeys= 12.to(14).toSeq
val batchGet2= spark.sqlContext
.read
.format("com.aerospike.spark.sql")
.option("aerospike.seedhost",AS_HOST)
.option("aerospike.featurekey", "/etc/aerospike/features.conf") 
.option ("aerospike.namespace", "test")
.option("aerospike.set", "input_data")
.option("aerospike.keyType", "int") //used to hint primary key(PK) type when inferred without schema.
.load.where((col("__key") isin (somePrimaryKeys:_*)) || ( col("__key") isin (someMoreKeys:_*) ))
batchGet2.show(5)
//We should got in total 13 records.

+-----+--------------------+--------+------------+-----+------+---+------+---+
|__key|            __digest|__expiry|__generation|__ttl|  name|age|salary| id|
+-----+--------------------+--------+------------+-----+------+---+------+---+
|   10|[16 50 E2 C7 BC 2...|       0|           1|   -1|name10| 10| 55249| 10|
|   13|[9C 90 67 F0 7F E...|       0|           1|   -1|name13| 13| 71789| 13|
|    7|[D3 C2 5B BE 77 3...|       0|           1|   -1| name7|  7| 55726|  7|
|    9|[23 B3 1A E8 CB 0...|       0|           1|   -1| name9|  9| 78500|  9|
|    6|[DD E4 52 09 AB 8...|       0|           1|   -1| name6|  6| 72064|  6|
+-----+--------------------+--------+------------+-----+------+---+------+---+
only showing top 5 rows



somePrimaryKeys: scala.collection.immutable.Range = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
someMoreKeys: scala.collection.immutable.Range = Range(12, 13, 14)
batchGet2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [__key: int, __digest: binary ... 7 more fields]


## Queries that do not include Primary Key in the Predicate

In [27]:

val somePrimaryKeys= 1.to(10).toSeq
val scanQuery1= spark.sqlContext
.read
.format("com.aerospike.spark.sql")
.option("aerospike.seedhost", AS_HOST)
.option ("aerospike.namespace", "test")
.option("aerospike.featurekey", "/etc/aerospike/features.conf") 
.option("aerospike.set", "input_data")
.option("aerospike.keyType", "int") //used to hint primary key(PK) type when inferred without schema.
.load.where((col("__key") isin (somePrimaryKeys:_*)) || ( col("age") >50 ))

scanQuery1.show()

//Since there is OR between PKs and Bin. It will be treated as Scan query. 
//Primary keys are not stored in bins(by default), hence only filters corresponding to bins are honored.  

+-----+--------------------+--------+------------+-----+-------+---+------+---+
|__key|            __digest|__expiry|__generation|__ttl|   name|age|salary| id|
+-----+--------------------+--------+------------+-----+-------+---+------+---+
| null|[0A 60 1D 97 98 5...|       0|           1|   -1|name486| 86| 74301|486|
| null|[0D 60 A3 4C 0C C...|       0|           1|   -1|name759| 59| 60474|759|
| null|[14 40 BB E5 AC F...|       0|           1|   -1|name796| 96| 89250|796|
| null|[15 70 45 1B 30 7...|       0|           1|   -1|name654| 54| 76596|654|
| null|[15 10 82 D6 73 0...|       0|           1|   -1|name961| 61| 78458|961|
| null|[29 90 F4 A2 39 9...|       0|           1|   -1| name64| 64| 64370| 64|
| null|[57 00 4F E5 E7 D...|       0|           1|   -1|name793| 93| 98266|793|
| null|[59 A0 08 B1 16 1...|       0|           1|   -1|name474| 74| 80971|474|
| null|[5D F0 D3 FE E9 1...|       0|           1|   -1|name454| 54| 63767|454|
| null|[6E F0 D2 3A B3 4...|       0|   

somePrimaryKeys: scala.collection.immutable.Range = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
scanQuery1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [__key: int, __digest: binary ... 7 more fields]


## Query with CDT

In [28]:
//Find all people who have atleast 5 jobs in past.
loadedComplexDFWithSchema
.withColumn("past_jobs", col("work_history.company_name"))
.withColumn("num_jobs", size(col("past_jobs")))
.where(col("num_jobs")  >4).show()

+--------------------+-----------+--------------------+--------------------+--------------------+--------+
|                name|        SSN|        home_address|        work_history|           past_jobs|num_jobs|
+--------------------+-----------+--------------------+--------------------+--------------------+--------+
|[Jamie, [[Patrici...|569-31-4715|[[53379, [James I...|[[Brown, Miller a...|[Brown, Miller an...|       5|
|[Michael, [[Micha...|455-56-8642|[[2300, [Bauer Ov...|[[Harrington, All...|[Harrington, Alle...|       5|
|[Luis, [[David, G...|818-16-1742|[[60659, [Oneill ...|[[Moss-Johnson, [...|[Moss-Johnson, St...|       5|
|[Tami, [[Joseph, ...|001-49-0685|[[23288, [Clark V...|[[Roberts PLC, [4...|[Roberts PLC, Hub...|       5|
|[Krista, [[Robert...|756-24-3462|[[64750, [Thomas ...|[[Baker PLC, [468...|[Baker PLC, Kirk ...|       5|
|[Kristina, [[Vick...|545-62-3152|[[70288, [Rebecca...|[[Vaughn Inc, [20...|[Vaughn Inc, Brow...|       5|
|[Elizabeth, [[And...|394-89-8545|[[4

## Use Aerospike Spark Connector Configuration properties in the Spark API to improve performance

aerospike.partition.factor: number of logical aerospike partitions [0-15]
aerospike.maxthreadcount : maximum number of threads to use for writing data into Aerospike
aerospike.compression : compression of java client-server communication
aerospike.batchMax : maximum number of records per read request (default 5000)
aerospike.recordspersecond : same as java client

#### Other
aerospike.keyType : Primary key type hint for schema inference. Always set it properly if primary key type is not string

See https://www.aerospike.com/docs/connect/processing/spark/reference.html for detailed description of the above properties